In [7]:
import glob
import os
import cPickle as pickle
import numpy as np
from collections import defaultdict
import pandas as pd
import gspread

import json
from oauth2client.client import SignedJwtAssertionCredentials

#Load encode manifest

In [8]:
json_key = json.load(open("../public clip-588adbc137f3.json"))
scope = ['https://spreadsheets.google.com/feeds']

credentials = SignedJwtAssertionCredentials(json_key['client_email'], json_key['private_key'], scope)
gc = gspread.authorize(credentials)

sht1 = gc.open_by_url("https://docs.google.com/spreadsheets/d/1ZU2mQh54jentqvhR_oMnviLGWR8Nw_x338gULzKjNDI/edit#gid=0")
ws = sht1.worksheet("Sheet1")
list_of_lists = ws.get_all_values()
manifest = pd.DataFrame(list_of_lists[1:], columns=list_of_lists[0])
manifest['qc_id'] = manifest.apply(lambda x: "{}_{}".format(x.ENCODE_ID, x.RBP), axis=1)
manifest.is_encode = manifest.is_encode == "TRUE"

#Make qsub command to parse the data

In [9]:
datasets = glob.glob("/projects/ps-yeolab2/encode/analysis/encode_v12/*.merged.r2.peaks.bed.clip_analysis.pickle")

In [24]:
def eplouge(name,count):
    return """#!/bin/bash
#PBS -N {0}
#PBS -l nodes=1:ppn=1
#PBS -o {0}.out
#PBS -e {0}.err
#PBS -V
#PBS -q home-yeo
#PBS -W group_list=yeo-group
#PBS -t 1-{1}
#PBS -l walltime=1:00:00
cd /home/gpratt/projects/encode/analysis/parsed_clip_analysis/
echo "hello, starting"
""".format(name, count)

prolouge = "eval ${cmd[$PBS_ARRAYID]}"

In [25]:

idr_cmds = [] 
for dataset in datasets:
    idr_cmds.append("python /home/gpratt/clipper/clipper/src/parse_clip_analysis.py -i {}".format(dataset))

In [26]:
total = 0
result = []
for cmd in idr_cmds:
    total += 1
    result.append('cmd[{}]="{}"'.format(total, cmd))

with open(os.path.join("/home/gpratt/projects/encode/scripts", "parse_clip_analysis.sh"), 'w') as out_file:
    out_file.write(eplouge("parse_clip_analysis", total))
    for line in result:
        out_file.write(line + "\n\n")
    out_file.write(prolouge + "\n")

#Actually read in interesting stuff

In [26]:
submitted_rbps = pd.read_excel("/home/gpratt/Dropbox/NatMethodsPaper/clustergram/ENCODE_submittedlist_20160104.xlsx", "Sheet1")

In [42]:
encode_manifest = manifest[manifest.is_encode]
encode_manifest['uID'] = encode_manifest.ENCODE_ID.apply(lambda x: x.split("_")[0])
encode_manifest = encode_manifest[encode_manifest.uID.isin(submitted_rbps.uID)]
hepg2_encode = encode_manifest[encode_manifest.cell_type == "HepG2"]
k562_encode = encode_manifest[encode_manifest.cell_type == "K562"]

/home/gpratt/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [11]:
parsed_files = glob.glob("/home/gpratt/projects/encode/analysis/parsed_clip_analysis/*")

In [12]:
result = {}

for fn in parsed_files:
    rbp = pd.read_csv(fn, index_col=[0,1])
    rbp.index.names = ['region', 'region_type']
    result[os.path.basename(fn).split(".")[0]] = rbp

result = pd.concat(result, names=['rbp', 'region', 'region_type'])

In [43]:
all_encode = result[result.index.get_level_values(level=0).isin(encode_manifest.qc_id)]
hepg2_encode = result[result.index.get_level_values(level=0).isin(hepg2_encode.qc_id)]
k562_encode = result[result.index.get_level_values(level=0).isin(k562_encode.qc_id)]

In [44]:
all_encode_premrna = all_encode.xs("total", level="region_type").xs("genes", level="region").T
hepg2_encode_premrna = hepg2_encode.xs("total", level="region_type").xs("genes", level="region").T
k562_encode_premrna = k562_encode.xs("total", level="region_type").xs("genes", level="region").T

all_encode_mrna = all_encode.xs("total", level="region_type").xs("exons", level="region").T
hepg2_encode_mrna = hepg2_encode.xs("total", level="region_type").xs("exons", level="region").T
k562_encode_mrna = k562_encode.xs("total", level="region_type").xs("exons", level="region").T

In [45]:
all_encode_premrna.to_csv("/home/gpratt/Dropbox/NatMethodsPaper/clustergram/all_premrna_encode.csv", index=False, sep="\t")
hepg2_encode_premrna.to_csv("/home/gpratt/Dropbox/NatMethodsPaper/clustergram/hepg2_premrna_encode.csv", index=False, sep="\t")
k562_encode_premrna.to_csv("/home/gpratt/Dropbox/NatMethodsPaper/clustergram/k562_premrna_encode.csv", index=False, sep="\t")

In [46]:
all_encode_mrna.to_csv("/home/gpratt/Dropbox/NatMethodsPaper/clustergram/all_mrna_encode.csv", index=False, sep="\t")
hepg2_encode_mrna.to_csv("/home/gpratt/Dropbox/NatMethodsPaper/clustergram/hepg2_mrna_encode.csv", index=False, sep="\t")
k562_encode_mrna.to_csv("/home/gpratt/Dropbox/NatMethodsPaper/clustergram/k562_mrna_encode.csv", index=False, sep="\t")

In [47]:
len(k562_encode_premrna.columns)

65

In [49]:
k562_encode_premrna

rbp,220_01_IGF2BP1,220_02_IGF2BP1,224_01_HNRNPM,224_02_HNRNPM,226_01_IGF2BP2,226_02_IGF2BP2,236_01_FMR1,236_02_FMR1,237_01_FXR2,237_02_FXR2,...,332_01_RBM27,332_02_RBM27,338_01_ILF3,338_02_ILF3,339_01_TAF15,339_02_TAF15,341_01_DGCR8,341_02_DGCR8,342_01_GTF2F1,342_02_GTF2F1
0.0,1.549855,0.810636,5.057505,5.773341,5.947895,4.380613,8.157844,5.616334,6.987390,6.074053,...,2.321764,3.687911,0.737396,0.859799,1.679097,1.839377,3.725600,4.754134,7.177299,5.508867
0.01,0.789474,0.615022,3.629355,4.090315,2.075972,1.839672,2.711758,2.214251,2.152372,2.192482,...,1.463415,2.236335,0.794558,0.949778,1.624932,1.899883,2.655318,3.079236,3.889652,3.358258
0.02,0.587146,0.484061,2.466613,2.724801,1.218314,1.048600,1.437475,1.240257,1.209821,1.233052,...,1.252345,1.537764,0.897451,0.866464,1.279113,1.468275,1.895928,2.258965,2.311581,2.309423
0.03,0.480032,0.363046,1.992670,2.268247,0.815142,0.763339,1.114564,0.942264,0.871469,0.878747,...,1.027205,0.993423,0.943181,0.839804,1.083288,1.335162,1.294531,1.580417,1.803092,1.829672
0.04,0.487966,0.412778,1.670388,1.840745,0.696389,0.727681,0.899290,0.862247,0.729304,0.708610,...,0.886492,1.070538,1.003201,0.996434,1.070789,1.230285,1.340401,1.481641,1.472866,1.670858
0.05,0.408622,0.371334,1.453427,1.664349,0.586433,0.513735,0.715265,0.678761,0.580032,0.661253,...,0.769231,0.907235,0.948897,1.023095,1.079122,1.335162,1.100861,1.193902,1.312137,1.366464
0.06,0.439037,0.359730,1.415512,1.597941,0.514595,0.450343,0.663183,0.645651,0.553020,0.613895,...,0.745779,0.775686,0.988910,1.103076,1.091621,1.185914,0.958157,0.983466,1.160174,1.293674
0.07,0.415234,0.411120,1.329149,1.498329,0.483807,0.466191,0.621517,0.623577,0.503263,0.582323,...,0.694184,0.811976,1.160398,1.023095,1.049956,1.072970,0.932674,1.013528,1.095882,1.217575
0.08,0.375562,0.353099,1.185912,1.270052,0.447155,0.434496,0.552074,0.554598,0.487625,0.506902,...,0.605066,0.675890,0.946039,0.946446,0.983292,1.060869,0.856225,1.000644,1.022824,1.042218
0.09,0.416556,0.421067,1.175380,1.282504,0.472078,0.451664,0.539921,0.554598,0.481938,0.533212,...,0.600375,0.666818,1.006059,1.026427,0.891630,1.097172,0.800163,0.901868,0.987755,1.052144


In [48]:
len(hepg2_encode_premrna.columns)

52